# RUN

In [1]:
import os
import pandas as pd
from modules.data_pipeline import process_data, get_path, export_file

# export_file.export_to_csv('21', 'igv')

# dict = process_data.read(99, 'igv')
paths = get_path.get(21,'igv')


{'ICA': {'ica0523': ['c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W21\\ica0523\\AT01ErrorHistory.csv',
   'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W21\\ica0523\\AT01TaskHistory.csv',
   'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W21\\ica0523\\AT02Data.csv',
   'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W21\\ica0523\\AT02ErrorHistory.csv',
   'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W21\\ica0523\\AT02TaskHistory.csv',
   'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W21\\ica0523\\AT06Data.csv',
   'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W21\\ica0523\\AT06ErrorHistory.csv',
   'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W21\\ica0523\\AT06TaskHistory.csv']},
 'TS': {'TS20240521': ['c:\\Use

# Test all weekly file

In [2]:
import os
import pandas as pd
import numpy as np

def get_all_weekly_file():
    root = 'c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99'
    pathlist = []
    for item in os.listdir(root):
        p = os.path.join(root, item)
        if os.path.isfile(p):
            pathlist.append(p)
    return pathlist

def lmd_rtg_operation(location_ref=str, task_stage=str):
    if ('YARD' in location_ref) & (task_stage=='Waiting for Operation'):
        return 'Waiting for Operation'
    else:
        return np.NaN

li = get_all_weekly_file()

In [3]:
for p in li:
    print(p)

    df = pd.read_csv(p, encoding='utf-8-sig', index_col=0)

    print(df.columns)
    print()

c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_CK.csv
Index(['vehicle_id', 'position_x', 'position_y', 'position_z', 'heading',
       'speed', 'speed_command', 'mileage_distance', 'power_on_time', 'soc',
       'fuel_level', 'local_time', 'chassis_mode', 'estop', 'charge_status',
       'task_state_running', 'task_state_estop', 'task_stage', 'current_task',
       'error_code', 'vehicle_mode', 'project', 'week'],
      dtype='object')

c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_DL.csv
Index(['vehicle_id', 'position_x', 'position_y', 'position_z', 'heading',
       'speed', 'speed_command', 'mileage_distance', 'power_on_time', 'soc',
       'fuel_level', 'local_time', 'chassis_mode', 'estop', 'charge_status',
       'task_state_running', 'task_state_estop', 'task_state_lock',
       'task_stage', 'current_task', 'error_code', 'target_location',
       'vesselVisitID', 'mission_type', 'container1_typ

In [ ]:
df.columns


In [ ]:
df['task_stage'].value_counts().index.tolist()

In [ ]:
path = os.path.abspath('c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W17\Igv_W17_TPY.csv')

d = pd.read_csv(path, encoding='utf-8-sig')
d.columns

In [ ]:
def lambda_power_usage(soc_diff):
    if soc_diff > 0 and soc_diff<=5: return soc_diff
    elif soc_diff<=0 and soc_diff>-5: return 0
    elif soc_diff >5: return 5
    elif soc_diff<=-5: return -1


for vehicle, d2 in d.groupby('vehicle_id'):
    for cycle, data in d2.groupby('Cycle Tag'):
        data['soc_diff'] = data['soc'] - data['soc'].shift(-1)
        data['power_usage'] = data['soc_diff'].apply(lambda x: lambda_power_usage(x))
        print( data['soc_diff'].value_counts().index.tolist(),
              data['power_usage'].value_counts().index.tolist())
        print()

# CKP info extract

In [ ]:
import pandas as pd
# display('C:\\Users\westwell\Documents\\.WorkDocuments\github\westwell_da\data\processed\W18\ICA\IgvData_ica20240504.csv')

p = 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W18\\ICA\\IgvData_ica20240503.csv'

d = pd.read_csv(p, index_col=0)
display(d.columns)

def feature_extraction(group):
    duration = group['duration_min'].sum()  # Example feature: sum of duration
    vehicle  = group['vehicle_id'].min()
    task_type= group['mission_type_org'].min()
    start    = group['local_time'].min()
    end      = group['local_time'].max()
    ck1      = group['local_time']

    return pd.Series({'duration_min': duration, 
                      'AT ID': vehicle,
                      'mission_type': task_type,
                      'cycle_start':start,
                      'cycle_end':  end,
                      })


grouped = d.groupby('Cycle Tag').apply(feature_extraction)

display(grouped.head(1))

# Group by 'Cycle Tag' and 'ckp_demo' and find the min of 'local_time'
g = d.groupby(['Cycle Tag', 'ckp_demo'])['local_time'].min().reset_index()

# Pivot the DataFrame
pivoted = g.pivot(index='Cycle Tag', columns='ckp_demo', values='local_time').reset_index()
display(pivoted.head(1))

merged = pd.merge(grouped, pivoted, on='Cycle Tag', how='left')
# merged.to_csv('ckp_info_export_0503.csv',date_format="%Y-%m-%d %H:%M:%S")
display(merged)

In [5]:
pip install tableauserverclient


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flair 0.13.1 requires urllib3<2.0.0,>=1.0.0, but you have urllib3 2.0.7 which is incompatible.
pyppeteer 1.0.2 requires urllib3<2.0.0,>=1.25.8, but you have urllib3 2.0.7 which is incompatible.



   ---------------------------------------- 0.0/137.7 kB ? eta -:--:--
   -- ------------------------------------- 10.2/137.7 kB ? eta -:--:--
   -------------- ------------------------ 51.2/137.7 kB 650.2 kB/s eta 0:00:01
   ---------------------------- --------- 102.4/137.7 kB 837.8 kB/s eta 0:00:01
   -------------------------------------- 137.7/137.7 kB 811.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/124.2 kB ? eta -:--:--
   --------- ------------------------------ 30.7/124.2 kB ? eta -:--:--
   --------- ------------------------------ 30.7/124.2 kB ? eta -:--:--
   ---------------------------------------  122.9/124.2 kB 1.2 MB/s eta 0:00:01
   -------------------------------------- 124.2/124.2 kB 808.3 kB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.16
    Uninstalling urllib3-1.26.16:
      Successfully uninstalled urllib3-1.26.16
